In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb
from scipy import stats

In [2]:
train_data = pd.read_csv("Train.csv")
test_data = pd.read_csv("Test.csv")
train_dummy = pd.get_dummies(train_data.drop(['uniqueid'], axis=1))
test_dummy = pd.get_dummies(test_data.drop(['uniqueid'], axis=1))

In [3]:
train_data.shape

(23524, 13)

In [4]:
ntrain = train_data.shape[0] 
ntest = test_data.shape[0]
dataset = pd.concat((train_data, test_data), sort=False).reset_index(drop=True)

In [5]:
dataset.shape

(33610, 13)

In [6]:
dataset.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [12]:
data = pd.get_dummies(dataset.drop(['uniqueid'], axis=1))
data_1 = data #For checking distributions

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()

In [14]:
data_1.head()

,year,household_size,age_of_respondent,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda,bank_account_No,bank_account_Yes,location_type_Rural,...,job_type_Dont Know/Refuse to answer,job_type_Farming and Fishing,job_type_Formally employed Government,job_type_Formally employed Private,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed
0,2018,3,24,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,2018,5,70,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
2,2018,5,26,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2018,5,34,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,2018,8,26,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
#dataset = LE.fit_transform(dataset)

In [15]:
data.shape

(33610, 42)

In [16]:
data.head()

,year,household_size,age_of_respondent,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda,bank_account_No,bank_account_Yes,location_type_Rural,...,job_type_Dont Know/Refuse to answer,job_type_Farming and Fishing,job_type_Formally employed Government,job_type_Formally employed Private,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed
0,2018,3,24,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,2018,5,70,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
2,2018,5,26,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2018,5,34,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,2018,8,26,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
data.describe()

,year,household_size,age_of_respondent,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda,bank_account_No,bank_account_Yes,location_type_Rural,...,job_type_Dont Know/Refuse to answer,job_type_Farming and Fishing,job_type_Formally employed Government,job_type_Formally employed Private,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed
count,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,...,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000,33610.000000
mean,2016.975930,3.791877,38.656114,0.257929,0.371318,0.281434,0.089319,0.601369,0.098542,0.610890,...,0.005385,0.232401,0.016453,0.044600,0.010324,0.237102,0.026867,0.044719,0.108123,0.274026
std,0.847353,2.223138,16.447127,0.437501,0.483164,0.449705,0.285207,0.489624,0.298051,0.487556,...,0.073188,0.422370,0.127213,0.206427,0.101084,0.425312,0.161697,0.206689,0.310540,0.446029
min,2016.000000,1.000000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,2.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2017.000000,3.000000,35.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2018.000000,5.000000,49.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2018.000000,21.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# SCALLING OF SOME COLUMNS
### Before scaling, the type of distribution must be checked first. This will help you know the scale that must be used

In [18]:
data_mean = np.mean(data_1[['household_size', 'age_of_respondent']])
data_median_hhsiz = np.median(data_1['household_size'])
data_median_age = np.median(data_1['age_of_respondent'])
data_mode = stats.mode(data_1[['household_size', 'age_of_respondent']])
print("The means for the various columns are: {} ".format(data_mean))
print("The median for the household_size is : {}".format(data_median_hhsiz))
print("The median for the Age_of_respondant is : {}".format(data_median_age))
print("The modes for the various columns are: {}".format(data_mode))

The means for the various columns are: household_size        3.791877
age_of_respondent    38.656114
dtype: float64 
The median for the household_size is : 3.0
The median for the Age_of_respondant is : 35.0
The modes for the various columns are: ModeResult(mode=array([[ 2, 30]], dtype=int64), count=array([[7501, 1499]]))


## Method 1.
### Min - Maximum Scale.
##### formular => Xnew = ((X-Xmin)/(Xmax - Xmin))
This scale is sensitive to outliers and its been used when the data is not Gaussian(Normal distribution)
To check normalisation, mean must be between mode & Median.
From the distribution of the two columns, the mean is greater than the mode and median. this indcates that the data is not Gaussian. 

In [19]:
from sklearn.preprocessing import MinMaxScaler
df = MinMaxScaler()
data[['household_size','age_of_respondent']] = df.fit_transform(data[['household_size','age_of_respondent']])

In [20]:
data.head()

,year,household_size,age_of_respondent,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda,bank_account_No,bank_account_Yes,location_type_Rural,...,job_type_Dont Know/Refuse to answer,job_type_Farming and Fishing,job_type_Formally employed Government,job_type_Formally employed Private,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed
0,2018,0.10,0.095238,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1
1,2018,0.20,0.642857,1,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
2,2018,0.20,0.119048,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,2018,0.20,0.214286,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,2018,0.35,0.119048,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


# 2. Standard Scaler:
### It assumes the data is normal distribution and its centered around 0, with a standard deviation of 1
z = (x - u) / s
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

## 3. Max Abs Scaler
#### Scale each feature by its maximum absolute value. This estimator scales and translates each feature individually such that the maximal absolute value of each feature in the training set is 1.0. It does not shift/center the data and thus does not destroy any sparsity.
On positive-only data, this Scaler behaves similarly to Min Max Scaler and, therefore, also suffers from the presence of significant outliers.
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

## 4) Robust Scaler
#### As the name suggests, this Scaler is robust to outliers. If our data contains many outliers, scaling using the mean and standard deviation of the data won’t work well.
This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile). The centering and scaling statistics of this Scaler are based on percentiles and are therefore not influenced by a few numbers of huge marginal outliers. Note that the outliers themselves are still present in the transformed data. If a separate outlier clipping is desirable, a non-linear transformation is required.
##### from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

## 5) Quantile Transformer Scaler
Transform features using quantiles information.
This method transforms the features to follow a uniform or a normal distribution. Therefore, for a given feature, this transformation tends to spread out the most frequent values. It also reduces the impact of (marginal) outliers: this is, therefore, a robust pre-processing scheme.
The cumulative distribution function of a feature is used to project the original values. Note that this transform is non-linear and may distort linear correlations between variables measured at the same scale but renders variables measured at different scales more directly comparable. This is also sometimes called as Rank scaler.
#### from sklearn.preprocessing import QuantileTransformer
scaler = QuantileTransformer()

## 6) Power Transformer Scaler
The power transformer is a family of parametric, monotonic transformations that are applied to make data more Gaussian-like. This is useful for modeling issues related to the variability of a variable that is unequal across the range (heteroscedasticity) or situations where normality is desired.
The power transform finds the optimal scaling factor in stabilizing variance and minimizing skewness through maximum likelihood estimation. Currently, Sklearn implementation of PowerTransformer supports the Box-Cox transform and the Yeo-Johnson transform. The optimal parameter for stabilizing variance and minimizing skewness is estimated through maximum likelihood. Box-Cox requires input data to be strictly positive, while Yeo-Johnson supports both positive or negative data.
#### from sklearn.preprocessing import PowerTransformer
scaler = PowerTransformer(method='yeo-johnson')

In [21]:
#Train and test datasets
train = data[:ntrain].copy()
test = data[ntrain:].copy()
test = test.reset_index(drop=True)

In [22]:
train.shape

(23524, 42)

In [23]:
test.shape

(10086, 42)

In [24]:
data.shape

(33610, 42)

In [ ]:
pip install rpy2